In [11]:
#get subset of MUSC records where journals equal those two chosen.

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
import pickle
import os
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
import datetime
from sklearn.metrics import roc_auc_score

In [10]:
#on VM
os.chdir('/mnt/mypartition/Desktop2/pubmed_nlp_research/DeepLearningMovies_datasets/')
import KaggleWord2VecUtility

In [2]:
#on local:
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [17]:
#one-time run
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [12]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Getting subset based on 2 journals (with abstracts and no Unicode encoding)

In [6]:
tree = ET.parse('/mnt/mypartition/Desktop2/testXMLwrite.xml')
root = tree.getroot()

In [7]:
#using this file for our example
xml_file = ET.parse('/home/sarahwie/Documents/zip/subset/zip/medline16n0189.xml')
root = xml_file.getroot()

This code subsets the records down to those with the titles we are looking for (as well as those with abstracts).

In [8]:
#make new XML file
rootNew = Element('MedlineCitationSet')
children = []

In [14]:
#replace titles here with those we're using
#titles = ['Physical review letters', 'Stroke; a journal of cerebral circulation']
titles = ['The Journal of biological chemistry','Journal of bacteriology']

i = 0
j = 0
k = 0
l = 0
for record in root.findall('MedlineCitation'):
    title = record.find('Article').find('Journal').find('Title').text
    
    if title in titles:
        j = j + 1
        #for test run, also check if abstract
        try:
            abstract = record.find('Article').find('Abstract').find('AbstractText').text
            if type(abstract) != str:
                i = i + 1
                print type(abstract)
            #append to subset file
            children.append(record)
            k = k + 1
        except AttributeError:
            #dont include if no abstract
            #something random to hold place of indent:
            l = l + 1

<type 'unicode'>
<type 'unicode'>


In [15]:
print i
print j
print k
print l

2
563
422
141


In [16]:
rootNew.extend(children)

#create tree and write to file
tree = ET.ElementTree(rootNew)
tree.write("/home/sarahwie/Documents/pubmed-nlp-research/output/subsetTwoJournals.xml")
        
pickle.dump(rootNew, open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/rootTwoJournals.p', 'wb'))

In [78]:
print i

2


In [17]:
#size of new subset
print len(rootNew.getchildren())
#verify that this is the correct number: yes

422


/home/sarahwie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  from ipykernel import kernelapp as app


### Convert XML to Pandas dataframe

In [1]:
#convert XML subset to pandas dataframe
def iter_docs(author):
    author_attr = author.attrib
    print author.attrib
    #titles_pos = ['Physical review letters']
    #titles_neg = ['Stroke; a journal of cerebral circulation']
    titles_pos = ['The Journal of biological chemistry']
    titles_neg = ['Journal of bacteriology']
    for record in author.findall('MedlineCitation'):
        doc_dict = author_attr.copy()
        print doc_dict
        #doc_dict.update(record.attrib)
        title = record.find('Article').find('Journal').find('Title').text
        doc_dict['journal'] = title
        abstract = record.find('Article').find('Abstract').find('AbstractText').text
        #make sure all abstracts are in string format to avoid later sentence parsing issues (69 in test run were unicode)
        if type(abstract) != str:
            abstract = abstract.encode('utf8')
        doc_dict['abstract'] = abstract
        #add sentiment label based on journal title
        if title in titles_neg:
            doc_dict['sentiment'] = 0
        elif title in titles_pos:
            doc_dict['sentiment'] = 1
        yield doc_dict
        
df = pd.DataFrame(list(iter_docs(rootNew)))
df = df[['sentiment', 'journal', 'abstract']]

#shuffle rows
df = df.iloc[np.random.permutation(len(df))]
df = df.reset_index(drop=True)
print df

NameError: name 'pd' is not defined

In [19]:
print df.loc[1][1]
#don't know why no quotation marks like when using pandas.to_csv in original w2v inversion
#but seems to work fine so ignore for now
df.loc[0:2]

Journal of bacteriology


,sentiment,journal,abstract
0,0,Journal of bacteriology,ompB mutants of Escherichia coli K-12 are mark...
1,0,Journal of bacteriology,Several lysosomal glycosidase activities were ...
2,0,Journal of bacteriology,The initiation of patulin biosynthesis in subm...


In [20]:
os.getcwd()

'/home/sarahwie/Documents/pubmed-nlp-research'

In [32]:
pickle.dump(df, open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/dfJournals.p', 'wb'))

### Get training and testing sets using 5-fold stratified cross-validation

In [21]:
df = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/dfJournals.p', 'rb'))
print len(df)

422


In [20]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

import pandas as pd # for quick summing within doc

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [22]:
a = datetime.datetime.now().replace(microsecond=0)

#5-fold stratified cross validation
from sklearn.cross_validation import StratifiedKFold
import numpy as np

#because no validation set, 4/5 of values go to train and 1/5 to test
#is this too high?***
#even though we shuffle, not as randomly distributed as the former method was
y = df['sentiment'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

i = 1
avg = []
for train_index, test_index in skf:
    print("ROUND", i)
    i = i + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]
    
    # ****** Split the training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    #here change to include all journal name labels of positive and negative
    inxs_pos = np.where(train1['sentiment'] == 1)[0].tolist()
    inxs_neg = np.where(train1['sentiment'] == 0)[0].tolist()

    print "Parsing sentences from training set"
    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer) 
        
    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences

    for review in train1["abstract"]:
        sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
    
    print "Building and training w2v models"
    ## create a w2v learner 
    basemodel = Word2Vec(
        workers=multiprocessing.cpu_count(), # use your cores
        iter=3, # iter = sweeps of SGD through the data; more is better
        hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
    basemodel.build_vocab(sentences) 
    
    #train models
    models = [deepcopy(basemodel) for y in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )
    
    print "Parsing test sentences"
    # read in the test set as a list of a list of words
    docs = []
    for review in test1["abstract"]:
        docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))
    
    print "scoring test set"
    # get the probs (note we give docprob our test set plus the models)
    probs = docprob(docs,models).astype(object)
    
    predictions = np.ones((probs.shape[0]))
    
    predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
    print predictions
    print np.size(predictions)
    
    score = roc_auc_score(test1["sentiment"], predictions)
    #score = np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)
    print score
    #append to average
    avg.append(score)


print("average of 5 rotations:", sum(avg)/float(len(avg)))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
85
0.511363636364
('ROUND', 2)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
85
0.5
('ROUND', 3)
Parsing sentences from tra

### Testing weighting_sentences_by_score method to get another result to compare- produces identical results again ugh

In [5]:
#cross out the line in docprob() function

In [19]:
a = datetime.datetime.now().replace(microsecond=0)

#5-fold stratified cross validation
from sklearn.cross_validation import StratifiedKFold
import numpy as np

#because no validation set, 4/5 of values go to train and 1/5 to test
#is this too high?***
#even though we shuffle, not as randomly distributed as the former method was
y = df['sentiment'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

d = 1
avg = []
for train_index, test_index in skf:
    print("ROUND", d)
    d = d + 1
    if d == 2:
        
        #use the indexes to subset the df pandas dataframe
        train1, test1 = df.iloc[train_index], df.iloc[test_index]

        # ****** Split the training set into clean sentences
        #
        sentences_pos = []  # Initialize an empty list of sentences
        sentences_neg = []  # Initialize an empty list of sentences

        #here change to include all journal name labels of positive and negative
        inxs_pos = np.where(train1['sentiment'] == 1)[0].tolist()
        inxs_neg = np.where(train1['sentiment'] == 0)[0].tolist()

        print "Parsing sentences from training set"
        for inx in inxs_pos:
            review = train1["abstract"].iloc[inx]
            sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

        for inx in inxs_neg:
            review = train1["abstract"].iloc[inx]
            sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer) 

        # ****** Split the labeled and unlabeled training sets into clean sentences
        #
        sentences = []  # Initialize an empty list of sentences

        for review in train1["abstract"]:
            sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

        print "Building and training w2v models"
        ## create a w2v learner 
        basemodel = Word2Vec(
            workers=multiprocessing.cpu_count(), # use your cores
            iter=3, # iter = sweeps of SGD through the data; more is better
            hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
            )
        basemodel.build_vocab(sentences) 

        #train models
        models = [deepcopy(basemodel) for y in range(2)]
        models[0].train(sentences_neg, total_examples=len(sentences_neg) )
        models[1].train(sentences_pos, total_examples=len(sentences_pos) )

        print "Parsing test sentences"
        # read in the test set as a list of a list of words
        docs = []
        for review in test1["abstract"]:
            docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

        print "scoring test set"
        # get the probs (note we give docprob our test set plus the models)
        probssentences = docprob(docs,models)
        
        probssentences["truelabel"] = 0
        for r in range(len(probssentences)):
            #get true label value from test1 where index matches doc number
            j = probssentences.iloc[r]['doc']
            probssentences.set_value(r, 'truelabel', test1.iloc[[j]]["sentiment"])
        
        #code to generate our arrays
        features = []
        labels = []

        import matplotlib
        import matplotlib.pyplot as plt
        for i in probssentences["doc"].unique():
            subset = probssentences.loc[probssentences["doc"] == i]
            #get label
            labels.append(subset['truelabel'].iloc[0])
            #always get the same column: negative
            probsneg = subset[[0]]
            (n, bins, patches) = plt.hist(probsneg, bins=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
            percentages = [x/sum(n) for x in n]
            #append feature vector to list
            features.append(percentages)
            
        from sklearn.ensemble import ExtraTreesClassifier
        from sklearn.feature_selection import SelectFromModel
        
        clf = ExtraTreesClassifier()
        clf = clf.fit(features, labels)
        weights = clf.feature_importances_  
        
        probssentences['0_weighted'] = 9.99
        probssentences['1_weighted'] = 9.99

        for i in range(len(probssentences)):
            valneg = probssentences.iloc[i][0]
            inxneg = 99999
            if 0 <= valneg < 0.1:
                inxneg = 0
            elif 0.1 <= valneg < 0.2:
                inxneg = 1
            elif 0.2 <= valneg < 0.3:
                inxneg = 2
            elif 0.3 <= valneg < 0.4:
                inxneg = 3
            elif 0.4 <= valneg < 0.5:
                inxneg = 4
            elif 0.5 <= valneg < 0.6:
                inxneg = 5
            elif 0.6 <= valneg < 0.7:
                inxneg = 6
            elif 0.7 <= valneg < 0.8:
                inxneg = 7
            elif 0.8 <= valneg < 0.9:
                inxneg = 8
            elif 0.9 <= valneg <= 1.0:
                inxneg = 9
            probssentences.set_value(i, '0_weighted', valneg * weights[inxneg])

            valpos = probssentences.iloc[i][1]
            inxpos = 9.9
            if 0 <= valpos < 0.1:
                inxpos = 0
            elif 0.1 <= valpos < 0.2:
                inxpos = 1
            elif 0.2 <= valpos < 0.3:
                inxpos = 2
            elif 0.3 <= valpos < 0.4:
                inxpos = 3
            elif 0.4 <= valpos < 0.5:
                inxpos = 4
            elif 0.5 <= valpos < 0.6:
                inxpos = 5
            elif 0.6 <= valpos < 0.7:
                inxpos = 6
            elif 0.7 <= valpos < 0.8:
                inxpos = 7
            elif 0.8 <= valpos < 0.9:
                inxpos = 8
            elif 0.9 <= valpos <= 1.0:
                inxpos = 9
            probssentences.set_value(i, '1_weighted', valpos * weights[inxpos])
            
        probs = probssentences.groupby("doc").sum()

        predictions = np.ones((probs.shape[0]))

        predictions[np.where(probs.iloc[:,0] >  probs.iloc[:,1])] = 0 # The first column is the negative model
        print predictions
        print np.size(predictions)

        score = roc_auc_score(test1["sentiment"], predictions)
        #score = np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)
        print score
        #append to average
        #avg.append(score)


#print("average of 5 rotations:", sum(avg)/float(len(avg)))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
scoring test set
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
85
0.511363636364
('ROUND', 2)
('ROUND', 3)
('ROUND', 4)
('ROUND', 5)
0:00:05


### Import KaggleWord2VecUtility since didn't work from file

In [7]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences